<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_03_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 3: Convolutional Neural Networks (CNN's)**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 3 Material

* Part 3.1: Using Convolutional Neural Networks
* **Part 3.2: Using Pre-Trained Neural Networks with Keras**
* Part 3.3: Facial Recognition and Analysis
* Part 3.4: Introduction to GAN's for Image and Data Generation

#### **Change your Runtime Now!**

For this lesson you must have a GPU hardware accelerator (e.g. `A100` if available).

## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [ ]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    Colab = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    Colab = False

You should see the following output except your GMAIL address should appear on the last line.

![__](https://biologicslab.co/BIO1173/images/class_01/class_01_6_image01A.png)

If your GMAIL address does not appear your lesson will **not** be graded.

## **TIME ALERT!**

This lesson will probably **require close to 3 hours to complete**. Besides the normal issues there are two instances in which you are required to train a neural network. Training time for both neural networks is about 1 hour.

Don't start working on this lesson if you don't have sufficient free time to finish it.

## Accelerated Run-time Check

You MUST run the following code cell to get credit for this class lesson. The code in this cell checks what hardware acceleration you are using. To run this lesson, you must be running a Graphics Processing Unit (GPU).

In [ ]:
# You must run this cell second

import tensorflow as tf

# ------------------------------------------------------------------------
# 0️⃣  Create check_device() function
# ------------------------------------------------------------------------

def check_device():
    # Check for available devices
    devices = tf.config.list_physical_devices()

    # Initialize device flags
    cpu = False
    gpu = False
    tpu = False

    # Check device types
    for device in devices:
        if device.device_type == 'CPU':
            cpu = True
        elif device.device_type == 'GPU':
            gpu = True
        elif device.device_type == 'TPU':
            tpu = True

    # Output device status
    if tpu:
        print("Running on TPU")
        print("WARNING: You must run this assigment using a GPU to earn credit")
        print("Change your RUNTIME now!")
    elif gpu:
        print("Running on GPU")
        gpu_info = !nvidia-smi
        gpu_info = '\n'.join(gpu_info)
        print(gpu_info)
        print("You are using a GPU hardware accelerator--You're good to go!")
    elif cpu:
        print("Running on CPU")
        print("WARNING: You must run this assigment using a GPU to earn credit")
        print("Change your RUNTIME now!")
    else:
        print("No compatible device found")
        print("WARNING: You must run this assigment using either a GPU or a TPU to earn credit")
        print("Change your RUNTIME now!")

# ------------------------------------------------------------------------
# 1️⃣  Call function
# ------------------------------------------------------------------------

check_device()

If you current `Runtime` is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_05/class_05_4_image11A.png)

However, if you received this warning message

![__](https://biologicslab.co/BIO1173/images/class_05/class_05_4_image14A.png)

You **MUST** go back and change your `Runtime` **NOW** before you continue.

### Create Custom Function

The cell below creates a custom function called `hms_string()`. This function is needed to record the time required to train your neural network model.

If you fail to run this cell now, you will receive one (or more) error message(s) later in this lesson.

In [ ]:
# Create custom function

# ------------------------------------------------------------------------
# 0️⃣  Create hms_string()
# ------------------------------------------------------------------------

# Simple function to print out elasped time
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

### **YouTube Introduction to ResNet**

Run the next cell to see short introduction to ResNet. This is a suggested, but optional, part of the lesson.

In [ ]:
from IPython.display import HTML
video_id = "nc7FzLiB_AY"
HTML(f"""
<iframe width="560" height="315"
  src="https://www.youtube.com/embed/{video_id}"
  title="YouTube video player"
  frameborder="0"
  allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture"
  allowfullscreen>
</iframe>
""")

### **DOWNLOAD AND INSTALL PRE-TRAINED NEURAL NETWORKS**

We will be using two pre-trained neural networks in this lesson, `ResNet50` and `ResNet101`. Run the next couple of code cells to download these neural networks to your Colab environment.

### Download `ResNet50`

In [ ]:
# Download ResNet50

import tensorflow.keras
from tensorflow.keras.applications import ResNet50

ResNet50_model_244 = ResNet50(weights='imagenet',include_top=True)

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image26C.png)

### Install Helper Functions for Examples and **Exercises**

The code in the cell below creates a two functions that we will need to use classify images in Examples 1 and 2.

* **make_square()** Since MobileNet is designed to classify images with the same number of horizontal and vertical pixels (i.e. a 'square' image), this function uses a combination of padding and cropping to convert any image into a 'square` image.

* **classify_image()** This function does most of the work. It first retrives the image from the HTTPS server and resizes it before processing it by the `ResNet50 model` that we previously downloaded. The actual prediction is made by this line of code:
```text
preds = ResNet50_model_244.predict(x)
```

In [ ]:
# Install helper functions

# %matplotlib inline   # uncomment if you are in a Jupyter notebook

# ----------------------------------------------------------------------
# Imports
# ----------------------------------------------------------------------
from PIL import Image, ImageFile, UnidentifiedImageError
import requests
import numpy as np
from io import BytesIO
from IPython.display import display

from tensorflow.keras.applications.resnet import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image

# ----------------------------------------------------------------------
# Global settings
# ----------------------------------------------------------------------
IMAGE_WIDTH, IMAGE_HEIGHT = 224, 224          # target resolution
ImageFile.LOAD_TRUNCATED_IMAGES = False       # keep the default behaviour

# Load the pretrained ResNet‑50 model
ResNet_model_244 = ResNet50(weights='imagenet')

# Base URL of the images
ROOT = "https://biologicslab.co/BIO1173/images/class_03/"

# ----------------------------------------------------------------------
# Utility functions
# ----------------------------------------------------------------------
def make_square(img):
    """
    Crop the image to a square (center‑aligned).
    """
    width, height = img.size
    side = min(width, height)
    left   = (width  - side) // 2
    top    = (height - side) // 2
    right  = left + side
    bottom = top  + side
    return img.crop((left, top, right, bottom))

# ----------------------------------------------------------------------
# Core function
# ----------------------------------------------------------------------
def classify_image(url):
    """
    Download an image from *url*, preprocess it, run it through ResNet‑50,
    display the image and print the top‑5 ImageNet predictions.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()          # raise an error on bad status
        img = Image.open(BytesIO(response.content)).convert('RGB')
    except UnidentifiedImageError:
        print("Error: Cannot identify image file. Check the image URL or file format.")
        return
    except Exception as e:
        print(f"Error while downloading or opening the image: {e}")
        return

    # Resize (or square‑crop first if you prefer)
    img = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.LANCZOS)

    # Preprocess
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Predict
    preds = ResNet50_model_244.predict(x)

    # Show the image
    display(img)

    # Print the top‑5 predictions
    print("\nTop‑5 predictions:")
    for pred in decode_predictions(preds, top=5)[0]:
        print(f"  {pred[1]:<25} : {pred[2]*100:5.2f}%")




If the code is correct you should **not** see any output.

### Example 1: Classify Images with ResNet50

The code in the cell below downloads an image of a dog from the course fileserver, https://biologicslab.co and then uses the pre-trained `ResNet50` neural network  `ResNet50` to to classify it.

The image name is "pembroke_corgi.jpg"

In [ ]:
# Example 1: Classify Image with ResNet50

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input

# Enter image name
image_name="pembroke_corgi.jpg"

# Generate image path
image_path=ROOT+image_name

# Print path
print(f"Analyzing ", image_path)

# Use ResNet50 to classify image
classify_image(image_path)

If the code is correct you should see the following output:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image25C.png)

`ResNet50` has been trained to recognize a wide range of common objectes. Here is a basic list:

| Domain                           | Example classes (just a few from each)                                   |
|----------------------------------|--------------------------------------------------------------------------|
| Animals – mammals                | chimpanzee, tiger, lion, zebra, elephant                                |
| Animals – birds                  | eagle, sparrow, penguin, flamingo, crane                                 |
| Animals – reptiles & amphibians  | alligator, snake, frog, turtle                                          |
| Animals – fish & crustaceans     | goldfish, salmon, shrimp, lobster                                       |
| Animals – insects                | bee, butterfly, ant, dragonfly                                          |
| Plants & flowers                 | daisy, sunflower, orchid, cactus                                        |
| Fruits & vegetables              | apple, orange, broccoli, carrot                                         |
| Vegetables & nuts                | potato, tomato, almond, cashew                                          |
| Instruments & musical equipment  | guitar, piano, violin, saxophone                                        |
| Sports & equipment               | tennis racket, golf ball, basketball, soccer ball                       |
| Vehicles – land                  | car, truck, motorcycle, bicycle                                         |
| Vehicles – air                   | airplane, helicopter, jet, glider                                       |
| Vehicles – water                 | boat, ship, submarine, ferry                                            |
| Buildings & architecture         | bridge, house, church, skyscraper                                       |
| Furniture                        | chair, table, sofa, bed                                                  |
| Home appliances                  | microwave, refrigerator, toaster, blender                               |
| Tools & hardware                 | hammer, screwdriver, wrench, drill                                      |
| Clothing & accessories           | t‑shirt, hat, shoes, glasses                                            |
| Food & drinks                    | coffee, tea, pizza, cake                                                |
| Miscellaneous                    | toilet paper, keyboard, watch, trophy                                   |


Clearly `ResNet50` was trained to classify dogs. What is somewhat interesting, is that `ResNet50` appears to be quite good as to correctly identify a dog's breed. `ResNet50` was absolutedly correct that the image showed a `Welsh Pembroke Corgi`.

The **Pembroke Welsh Corgi** is a spirited, compact herding dog originally from Wales. Known for its short legs, fox-like ears, and expressive eyes, it's affectionate, intelligent, and highly trainable. Pembrokes thrive on activity, enjoy family life, and are renowned for their loyal, playful nature and distinctive “corgi grin.”

### **Exercise 1: Classify Images with ResNet50**

In the cell below write the code to download an another dog image from the course fileserver:

```type
image_name="bouvier_des_flandres.jpg"
```
and then uses `ResNet50` to classify it. This species is much less common than Corgis.

The **Bouvier des Flandres**, or **Bouvier de Flanders**, is a muscular, medium‑to‑large herding dog originally bred in the Flemish region of Belgium to manage cattle, sheep and pack loads. With a dense, double‑coated coat that comes in black, brown, red or tricolor, they are built for endurance and can thrive in both wet and dry climates. Their temperament is confident and affectionate, yet they possess a strong work ethic and a naturally protective instinct, making them excellent companion animals as well as valuable in search‑and‑rescue, therapy, and police work. Bouviers are intelligent and trainable, but they require consistent socialization and mental stimulation to prevent stubbornness or frustration. Health concerns are relatively few—chiefly hip dysplasia, gastric dilatation‑volvulus (bloat) and certain eye conditions—so with proper care, they can live 10–12 years.

In [ ]:
# Insert your code for Exercise 1 here

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input

# Enter image name
image_name="bouvier_des_flandres.jpg"

# Generate image path
image_path=ROOT+image_name

# Print path
print(f"Analyzing ", image_path)

# Use ResNet50 to classify image
classify_image(image_path)

If the code is correct you should see the following output:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image27C.png)

This time `ResNet50` got a little confused. It was "pretty sure" (62%) that the dog in the image was an `Irish water spaniel`.

![___](https://biologicslab.co/BIO1173/images/class_03/IrishWaterSpaniel.jpg)

**Irish Water Spaniel**

However, `ResNet50` second guess (with a 29% certainty) was correct.

This mis-identification nonwithstanding, I suspect `ResNet50` is still probably better at identifying dog breeds than the average person.

### Example 2: Classify Retinal Image with ResNet50

What about medical image data? Can `ResNet50` analyze a **color fundus photograph** of the interior surface of a human retina?

Run the code in the Example 2 to see how `ResNet50` does with the following retinal image:

```text
"Retina_Score_0.png"
```


In [ ]:
# Example 2: Classify Retinal Image with ResNEt50

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input

# Enter image name
image_name="Retina_Score_0.png"

# Generate image path
image_path=ROOT+image_name

# Print path
print(f"Analyzing ", image_path)

# Use ResNet50 to classify image
classify_image(image_path)

If the code is correct you should see the following output:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image20C.png)


As expected, `ResNet50` was not trained to recognize retinal images. For this particular retinal image, `ResNet50` was pretty certain (84.5% probability) that this image was a lampshade. Here is an image of a lampshade that is vaguely similar:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image49C.png)

**Metal Pendant Lamp Shade Retro Style Lampshade for Kitchen Dining Room water drop - Walmart.com**

### **Exercise 2: Classify Retinal Image with `ResNet50`**

In the cell below write the code to analyze the another retinal color fundus image:
```text
"Retina_Score_1.png"
```


In [ ]:
# Insert your code for Exercise 2 here



If the code is correct you should see the following output:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image21C.png)

Once again, it would be charitable to say that `ResNet50` was a little weak when it comes to analyzing clinical images.

# **Transfer Learning for Computer Vision**

## **`ResNet`**

Many advanced prebuilt neural networks are available for computer vision, and Keras provides direct access to many networks. **Transfer Learning** is the technique where you use these prebuilt neural networks.

There are several different levels of transfer learning.

* Use a prebuilt neural network in its entirety
* Use a prebuilt neural network's structure
* Use a prebuilt neural network's weights

In this lesson we will use a popular prebuilt CNN called **`ResNet (Residual Network)`** built by Microsoft Research in 2015. The name comes from the fact that this network was designed to address the **vanishing gradient problem** that occurs when training very deep neural networks.

Instead of learning the direct mapping from input to output, `ResNet` learns the residual (i.e., the difference between the input and the output).
This is achieved using **skip connections** (also called shortcut connections), which allow the input to bypass one or more layers and be added directly to the output.

## **Transfer Learning**

**Transfer learning** is a machine learning technique where a model developed for one task is reused as the starting point for a model on a second task. In the context of image recognition, this approach is particularly effective when using deep convolutional neural networks (CNNs) like **`ResNet`**.

### **Why Use Transfer Learning?**

Training deep neural networks from scratch requires large datasets and significant computational resources. Transfer learning mitigates this by leveraging pre-trained models—typically trained on large benchmark datasets like ImageNet—to extract general features from images. These features can then be fine-tuned for a specific task with a smaller, domain-specific dataset.

### **How `ResNet` Supports Transfer Learning**

`ResNet` is a widely used CNN architecture known for its use of **residual connections**, which help in training very deep networks by addressing the vanishing gradient problem. Pre-trained versions of `ResNet` (e.g., `ResNet-50`, `ResNet-101`) are commonly used as feature extractors in transfer learning workflows.

### **Benefits**

- **Reduced Training Time**: Leverages existing learned features.
- **Improved Performance**: Often achieves better accuracy with less data.
- **Flexibility**: Can be adapted to a wide range of image classification tasks.

Transfer learning with `ResNet` is a powerful and efficient approach for developing high-performing image recognition models, especially when data or computational resources are limited.


# Example 3: Improving `ResNet` Classification Accuracy of Diabetic Retinopathy

We know from from above that the native (base) `ResNet50` neural network is unable to effectively analyze clinical retinal images.

Example 3 will demonstrate how **transfer learning** can leverage `ResNet50` as a starting point to create a new neural network that can effectively classify retinal images as to their degree of diabetic retinopathy.  

#### **`Diabetic Retinopathy Image Dataset` Classes**

The dataset we will be using is called the **`Diabetic Retinopathy Image Dataset`**. This image dataset consists of color fundus photographs—high‑resolution RGB images of the interior surface of the eye (the retina).

These are standard clinical retinal images obtained with a fundus camera (usually a 45° or 50° field-of-view, non‑mydriatic or mydriatic camera) and capture the posterior pole (macula, optic disc, retinal vessels, and surrounding retinal tissue). The images are typically stored as JPEG/PNG files with dimensions on the order of several thousand pixels (e.g., 3500x2333 px) and are used for grading the severity of diabetic retinopathy.

The severity of diabetic retinopathy was divided into **five classes** as follows:
1. **Class 0** - No Diabetic Retinopathy (No_DR)
2. **Class 1** - Mild Non-Proliferative Diabetic Retinopathy (NPDR)
3. **Class 2** - Moderate NPDR
4. **Class 3** - Severe NPDR
5. **Class 4** - Proliferative Diabetic Retinopathy (PDR)

An example of each class is shown below:

![__](https://biologicslab.co/BIO1173/images/class_03/DiabeticRetinopathyData.png)

It should be noted that differences in apparent retinal size has nothing to do with the degree of retinopathy but instead reflects the fact that the retinal images were taken by a large number of clinicians, with different imaging equipment and procedures. In other words, there is considerable "noise" in the image set.



### Install `keras_preprocessing` Package

As part of Example 3 and **Exercise 3** we need to install the Keras function `ImageDataGenerator` which is part of the Keras package `keras_preprocessing`.

Run the code in the next cell to install `keras_preprocessing` and its `ImageDataGenerator`.

In [ ]:
# Install Keras package

!pip install keras_preprocessing

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image06C.png)

### Example 3 - Step 1: Set ENVIRONMENTAL VARIABLES

The code in the cell below defines a number of `ENVIRONMENTAL VARIABLES` that are needed for later code cells.

The use of **Enivornment Variables** can allow code to be **configurable without modification**. For example, you might use different database URLs for development, testing, and production environments. As you will see later, you will re-use this code cell for **Exercise 3 - Step 1** below.

In [ ]:
# Example 3 - Step 1: Set ENVIRONMENTAL VARIABLES

import os

# ------------------------------------------------------------------------
# 1️⃣  Create variables for downloading loading Zip file
# ------------------------------------------------------------------------
URL = "https://biologicslab.co/BIO1173/data/"
DOWNLOAD_SOURCE = URL+"diabetic_retinopathy_train_244.zip"
DOWNLOAD_NAME = DOWNLOAD_SOURCE[DOWNLOAD_SOURCE.rfind('/')+1:]
print("DOWNLOAD_SOURCE=",DOWNLOAD_SOURCE)
print("DOWNLOAD_NAME=",DOWNLOAD_NAME)

# ------------------------------------------------------------------------
# 2️⃣  Create variables for extracting the Zip file
# ------------------------------------------------------------------------
PATH = "./"
EXTRACT_TARGET = os.path.join(PATH,"retinopathy_244")
SOURCE = os.path.join(EXTRACT_TARGET, "train_244")
print("EXTRACT_TARGET=",EXTRACT_TARGET)
print("SOURCE=",SOURCE)

# ------------------------------------------------------------------------
# 3️⃣  Print variables for debugging
# ------------------------------------------------------------------------
print("ENVIRONMENTAL VARIABLES were successfully created.")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image08C.png)

### Example 3 - Step 2: Download and Extract Image Data

The code in the cell below downloads a zip file and extracts it. The names of the file server, zip file and folders were set above as **`ENVIRONMENTAL VARIABLES`** in the previous step.

**TIME ALERT:** Even when compressed (i.e. "zipped"), image data file are typically quite large so download and extraction times can often take a few minutes.

In [ ]:
# Example 3 - Step 2: Download and Extract Image Data

import os
import subprocess
import sys

# --------------------------------------------------------------
# 1️⃣  Create directories
# --------------------------------------------------------------
print("Creating necessary directories...", end='')
# Create necessary directories
os.makedirs(SOURCE, exist_ok=True)
os.makedirs(EXTRACT_TARGET, exist_ok=True)
print("done.")

# --------------------------------------------------------------
# 2️⃣  Download Zip file
# --------------------------------------------------------------
print(f"Downloading {DOWNLOAD_NAME}...", end='')
# Define paths and URLs
download_path = os.path.join(PATH, DOWNLOAD_NAME)
extract_path = os.path.join(EXTRACT_TARGET, DOWNLOAD_NAME)
# Download the file
try:
    result = subprocess.run(
        ["wget", "-O", DOWNLOAD_NAME, DOWNLOAD_SOURCE],
        check=True,
        capture_output=True,
        text=True
    )
    print("done.")
except subprocess.CalledProcessError as e:
    print(f"Download failed with error: {e}")
    print(f"Error output: {e.stderr}")
    sys.exit(1)

# --------------------------------------------------------------
# 3️⃣  Extract Zip file
# --------------------------------------------------------------
print(f"Extracting {DOWNLOAD_NAME} to {EXTRACT_TARGET}...", end='')

# Check if zip file exists and has content
if not os.path.exists(DOWNLOAD_NAME):
    print(f"Error: Zip file {DOWNLOAD_NAME} does not exist")
    sys.exit(1)

if os.path.getsize(DOWNLOAD_NAME) == 0:
    print(f"Error: Zip file {DOWNLOAD_NAME} is empty")
    sys.exit(1)

# Extract the file with error handling
try:
    # Use -o flag to overwrite files without prompting
    # Use -q for quiet mode
    result = subprocess.run(
        ["unzip", "-o", "-q", DOWNLOAD_NAME, "-d", EXTRACT_TARGET],
        check=True,
        capture_output=True,
        text=True
    )
    print("done.")
except subprocess.CalledProcessError as e:
    print(f"Error: Unzipping failed with return code {e.returncode}")
    print(f"Error output: {e.stderr}")
    print(f"Command that failed: {e.cmd}")
    sys.exit(1)
except FileNotFoundError:
    print("Error: unzip command not found. Please install unzip.")
    sys.exit(1)

# --------------------------------------------------------------
# 4️⃣  Verify Extraction was successful
# --------------------------------------------------------------
print("Verifying Extraction...")
try:
    # Check if extraction directory exists
    if not os.path.exists(EXTRACT_TARGET):
        print(f"Error: Extraction directory {EXTRACT_TARGET} does not exist")
        sys.exit(1)

    # List contents to verify
    contents = os.listdir(EXTRACT_TARGET)
    if len(contents) == 0:
        print("Warning: Extraction directory is empty")
    else:
        print(f"Successfully extracted {len(contents)} items:")
        for item in sorted(contents)[:10]:  # Show first 10 items
            item_path = os.path.join(EXTRACT_TARGET, item)
            if os.path.isfile(item_path):
                size = os.path.getsize(item_path)
                print(f"  - {item} ({size} bytes)")
            else:
                print(f"  - {item} (directory)")

        if len(contents) > 10:
            print(f"  ... and {len(contents) - 10} more items")

    # Try to get more detailed information about what was extracted
    result = subprocess.run(
        ["ls", "-la", EXTRACT_TARGET],
        capture_output=True,
        text=True,
        check=False  # Don't raise exception for this command
    )
    if result.returncode == 0:
        print("Directory contents:")
        print(result.stdout)

except Exception as e:
    print(f"Error during verification: {e}")
    sys.exit(1)

print("Extraction completed successfully!")


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image09D.png)



### Example 3 - Step 3: Load Labels for the Training Set

CNNs are trained by _supervised_ learning, which means that during each training step the network's prediction must be compared to a known correct answer. The label supplies this target: it allows a loss function (e.g., cross‑entropy, IoU, etc.) to be computed, which in turn provides gradients that tell the optimizer how to adjust the weights. Without a label, the loss cannot be evaluated, no gradients can be derived, and the network has no signal to improve its predictions. Thus, labels are essential for defining the training objective and enabling the network to learn from data.

The file `trainLabels.csv` contains the label information for our retinal images. This file has just two columns, **image** and **level**. The `image` specifies the image's filename and from which eye the image was obtained; for example, `10_left.png`. The `level` column contains the a numerical value between 0 and 4 which indicates the serverity of diabetic retinopathy. So in this example the **_level_** is the **image label**.

The code in the cell below reads the file `trainLabels.csv` and creates a Pandas dataframe called `eg_raw_df` to store the label information. A short amount of `eg_raw_df` is printed out for inspection to make sure the code worked as expected.

In [ ]:
# Example 3 - Step 3: Load the labels for the training set

import pandas as pd

# Read labels and create dataframe
eg_raw_df = pd.read_csv(
        os.path.join(EXTRACT_TARGET,"trainLabels.csv"),
        na_values=['NA', '?'])

# Add file extention
image_col = 'image'
eg_raw_df[image_col] = eg_raw_df[image_col].astype(str) + '.png'

# Print sample for verification
eg_raw_df

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image04C.png)

### Example 3 - Step 4: Validate Images

The code in the cell below checks to see if there is an actual retinal image for each image label in contained the DataFrame `eg_raw`, that was generated in the previous step.  

In [ ]:
# Example 3 - Step 4: Validate Images

from pathlib import Path

# Validate that all image files actually exist
def validate_image_files(df, source_path):
    source = Path(source_path)
    existing_files = set()

    # Get all actual files in the directory
    for file_path in source.rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
            existing_files.add(file_path.name)

    # Check which files in DataFrame actually exist
    df['file_exists'] = df['image'].apply(lambda x: x in existing_files)

    print(f"Total images in DataFrame: {len(df)}")
    print(f"Images that exist: {df['file_exists'].sum()}")
    print(f"Missing images: {(~df['file_exists']).sum()}")

    # Filter to only include existing files
    valid_df = df[df['file_exists']].copy()
    print(f"Valid DataFrame size: {len(valid_df)}")

    return valid_df

# Use it:
eg_raw_df = validate_image_files(eg_raw_df, SOURCE)

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image24D.png)

### Example 3 - Step 5: Split Images into Training and Validation Sets

If you judge the model only on the same data you used to update its weights, you’ll get an *optimistic* estimate of its performance – the model will look perfect on the training set but will usually fail on new images.  
Splitting the data into **training + validation** lets you:

1. **Measure true generalization** - see how the network behaves on unseen samples.  
2. **Tune hyper‑parameters** - learning rate, number of layers, data‑augmentation policies, etc.  
3. **Detect & prevent over‑fitting** - monitor validation loss/accuracy during training and stop or adjust when the model starts to degrade.  
4. **Select the best model** - keep the checkpoint that had the lowest validation error.  

Without a validation set, you'll be blind to over-fitting, you'll have no reliable metric for early stopping, and you'll have no principled way to pick the best architecture or hyper-parameters.

The code in the cell below, splits the retinal images into a training set and a validation set. How much of the train set is used for the validation set depends on the variable `FRAC`. In the cell below, `FRAC` is set to 0.8 which means 80% of the images will be put into the training set (`eg_train_df`) and the remaining 20% will be put into the validation set (`eg_val_df`). Which images are used in each set is randomize. It should be noted that only the **image names** are being "split" into a training and validation pool. Later an image generator will use these two DataFrames to actually generate two image sets.

After the split, the number images assigned to both sets is printed out.

In [ ]:
# Example 3 - Step 5: Split Images into Training and Validation Sets

# Set split fraction
FRAC=0.8  # 80% training / 20% validation

# Convert the class column to string – required for `flow_from_dataframe`
eg_raw_df['level'] = eg_raw_df['level'].astype(str)

# ------------------------------------------------------------------
#  1️⃣ Randomly split data into training and validation sets
# ------------------------------------------------------------------
eg_train_df = eg_raw_df.sample(frac=FRAC, random_state=42)
eg_val_df   = eg_raw_df.drop(eg_train_df.index)

# Calculate the split fraction as sanity check
split_fraction = len(eg_train_df) / (len(eg_val_df) + len(eg_train_df))

# Print out numbers
print(f"Training set size   : {len(eg_train_df)}")
print(f"Validation set size : {len(eg_val_df)}")
print(f"Calculated split fraction =", split_fraction)

# Quick sanity check
print("\nSample training rows:")
print(eg_train_df[['image', 'level']].head())

print("\nSample validation rows:")
print(eg_val_df[['image', 'level']].head())

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image25D.png)

### Example 3 - Step 6: Final Check for Valid Images

The code in the cell below does one more final check for valid image files before the `Image Generator` is created in the next step.

In [ ]:
# Example 3 - Step 6: Final Check for Valid Images

import os
from pathlib import Path

# Check which files are actually valid
def check_valid_images(df, source_path):
    valid_files = []
    invalid_files = []

    for filename in df['image']:
        file_path = Path(source_path) / filename
        if file_path.exists() and file_path.is_file():
            valid_files.append(filename)
        else:
            invalid_files.append(filename)

    print(f"Total files: {len(df)}")
    print(f"Valid files: {len(valid_files)}")
    print(f"Invalid files: {len(invalid_files)}")

    # Remove invalid files from your dataframe
    df_clean = df[df['image'].isin(valid_files)]
    print(f"After cleaning: {len(df_clean)} samples")

    return df_clean

# Apply cleaning to both train and validation sets
eg_train_df = check_valid_images(eg_train_df, SOURCE)
eg_val_df = check_valid_images(eg_val_df, SOURCE)


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image22D.png)

### Example 3 - Step 7: Create Image Generator

The code in the cell below uses the `TensorFlow/Keras` utility `ImageDataGenerator` to creates an **Image Generator**.  

**`ImageDataGenerator`** is a powerful utility in Keras that allows you to efficiently load, preprocess, and augment image data for training deep learning models. It helps improve model generalization by applying random transformations to training images, such as rotations, shifts, flips, and more.

#### **Key Features**
- **Real-time data augmentation**: Applies transformations on the fly during training.
- **Memory efficiency**: Loads images in batches, reducing memory usage.
- **Preprocessing**: Includes rescaling, normalization, and standardization.

#### **Common Parameters**
```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values
    rotation_range=40,           # Random rotation
    width_shift_range=0.2,       # Horizontal shift
    height_shift_range=0.2,      # Vertical shift
    shear_range=0.2,             # Shear transformation
    zoom_range=0.2,              # Zoom in/out
    horizontal_flip=True,        # Flip images horizontally
    fill_mode='nearest'          # Fill in new pixels
)
```
**IMPORTANT NOTE:**

You need to be very careful to correctly set the **IMAGE SIZE**. This is especially true when performing transfer learning with pre-trained neural networks like `ResNet50`. `ResNet50` is designed to work with square images of exactly `244` pixels wide and `244` pixels high (i.e. `244 X 244`) that have 3 color channels (i.e. `RGB`) and no `alpha` channel.

In the cell below, here is the code that specifies the image size:

```type
# Specify image size
IMG_W, IMG_H = 244, 244
```
You should also note that we set the `batch` sizes for training at this step in process:
```text
# Specify batch size
BATCH_TRAIN  = 64
BATCH_VAL    = 64
```
If we need to use a different batch size we will need to go back and re-run this cell again.

You should also note that we only augment the **training images** --  the **validation images** are **not** augmented.

In [ ]:
# Example 3 - Step 7: Create Image Generator

from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf

# Specify image size
IMG_W, IMG_H = 244, 244

# Specify batch size
BATCH_TRAIN  = 64
BATCH_VAL    = 64

# --------------------------------------------------------------------
# 1️⃣ Training generator – augmentations
# --------------------------------------------------------------------
eg_train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # for *ResNet* pre‑proc
    width_shift_range=0.2,       # Horizontal shift
    height_shift_range=0.2,      # Vertical shift
    horizontal_flip=True,        # Flip images horizontally
    zoom_range=0.2,              # Zoom in/out
    fill_mode='nearest',         # Fill in new pixels
    rotation_range=20,           # Add some rotation
)

# Optimize data loading with prefetching and caching
eg_train_gen = eg_train_datagen.flow_from_dataframe(
    dataframe=eg_train_df,  # Labels for training images
    directory=str(SOURCE),
    x_col='image',          # column that holds the file name
    y_col='level',          # column that holds the class string
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_TRAIN,
    class_mode='categorical',   # one‑hot (shape (batch, 5))
    shuffle=True,
    seed=42,                  # For reproducibility
)

# --------------------------------------------------------------------
# 2️⃣ Validation generator -- no augmentation
# --------------------------------------------------------------------
eg_val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

eg_val_gen = eg_val_datagen.flow_from_dataframe(
    dataframe=eg_val_df,   # Labels for the validation images
    directory=str(SOURCE),
    x_col='image',
    y_col='level',
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_VAL,
    class_mode='categorical',
    shuffle=False,
)

# Sanity Check
eg_x_train, eg_y_train = next(eg_train_gen)
eg_x_val,   eg_y_val   = next(eg_val_gen)

print("TRAIN batch  : ", eg_x_train.shape, eg_y_train.shape)
print("VAL   batch  : ", eg_x_val.shape,   eg_y_val.shape)

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image23D.png)

This indicates that we will be using 13,958 retinal images in our training set and 3,490 images in our validation set. Each image is `244` x `244` pixels with `3` color channels (RGB).

### Example 3 - Step 8: Check Class Distribution

The code in the cell below generates a histogram showing the distribution of the 5 classes in the validation data set.

#### **Why Check Class Distribution in CNN Classification?**

When training a Convolutional Neural Network (CNN) for image classification, examining the distribution of classes in your dataset helps ensure that your model learns effectively and generalizes well. Here's why it's important:

##### **1. Detecting Class Imbalance**
If one class has significantly more samples than others, the model may become biased toward predicting the majority class. This can lead to:
- High accuracy but poor performance on minority classes.
- Misleading evaluation metrics.

##### **2. Choosing the Right Metrics**
In imbalanced datasets, accuracy alone is not a reliable metric. You may need to use:
- Precision, recall, F1-score
- Confusion matrix
- ROC-AUC (for binary classification)

##### **3. Designing Better Validation Strategies**
Knowing the class distribution helps in:
- Stratified sampling for train/test splits
- Ensuring each class is represented in validation and test sets

##### **4. Applying Corrective Techniques**
If imbalance is detected, you can apply:
- **Data augmentation** for minority classes
- **Class weighting** in the loss function
- **Oversampling** or **undersampling**
- **Synthetic data generation** (e.g., SMOTE)

##### **5. Improving Interpretability**
Understanding class distribution helps interpret model behavior and debug issues like:
- Why the model is misclassifying certain classes
- Why training loss is low but validation performance is poor


### **Best Practice**
Always visualize class distribution before training using a bar chart or value counts.

Following **Best Practice**, the code in the cell below generates a bar chart showing the class distribution before we start our training.

In [ ]:
# Example 3 - Step 8: Check Class Distribution

import numpy as np
import matplotlib.pyplot as plt

# Count class distribution
eg_labels_val = eg_val_gen.classes
eg_class_counts = np.bincount(eg_labels_val)

# Plot distribution
plt.bar(range(len(eg_class_counts)), eg_class_counts)
plt.xlabel("Class Index")
plt.ylabel("Number of Samples")
plt.title("Validation Set Class Distribution")
plt.show()


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image04D.png)

The classes are not balanced. Healthy retinal images (Class `0`) outnumbers the other classes. Unfortunately this situation is quite common in medical imaging of pathological states. The problem is that our neural network model can quickly learn that it picks `0` as the image label, it will be correct _most_ of the time. We need to keep this imbalance in mind when we interpret the results of training our neural network.

### Example 3 - Step 9: Setup and Compile Neural Network

The code in the cell below demonstrates how to use **transfer learning** with the **ResNet50** architecture for a custom image classification task involving 5 classes (i.e. degrees of diabetic retinopathy).

#### **What It Does**

- **Loads a pretrained `ResNet50 model`** (trained on ImageNet) without its top classification layer.
- **Freezes the base model's weights** to retain learned features and prevent them from being updated during training.
- **Adds custom layers** on top of the base model:
  - Global average pooling to reduce spatial dimensions.
  - Dense layer with ReLU activation for learning task-specific features.
  - Dropout for regularization to prevent overfitting.
  - Final softmax layer for multi-class classification (5 classes).
- **Compiles the model** using the `Adam optimizer` and `categorical crossentropy loss`, suitable for multi-class classification.

#### **Use Case**

This approach is ideal when:
- You have limited training data.
- You want to leverage powerful pretrained models.
- You need to adapt a general-purpose model to a specific classification task.

The code in the cell first generates a `base` model from `ResNet50`. It then modifies this base by adding several input layers and an output layer that are tailored to meet our specific use. The finished model called `ResNet50_model_244`. Once out model `ResNet50_model_244` is assembled, it is compiled with the appropiate optimizer (`Adam`), loss function (`categorical_crossentropy`) and metric (`accuracy`) for a _classification_ neural network.

In [ ]:
# Example 3 - Step 9: Setup and Compile Neural Network

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers

# Setup base network
base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_H, IMG_W, 3))
base.trainable = False          # freeze layers

# Add our layers to the base
x = base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)                   # add drop out layer
predictions = layers.Dense(5, activation='softmax')(x)

# Create model
ResNet50_model_244 = models.Model(inputs=base.input, outputs=predictions)

# Use Adam optimizer
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile model
ResNet50_model_244.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'])

If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image33C.png)

### Example 3 - Step 10: Train Neural Network

The code in the cell below trains the neural network `ResNet50_model_244` for the number of epochs specified by the variable `EPOCHS`. A second variable, `PATIENCE`, controls the Early Stopping monitor. To keep training time reasonable the variable EPOCH has been set to 10 and the variable PATIENCE has been set to 3.

**TIME ALERT!**

We are analyzing a **LARGE** image dataset so training will take time. Even using Google fastest GPU hardware acceleration, it will probably take more than **1 hour to complete**. So don't start training if you can't afford to wait.

**DON'T WASTE YOUR MONEY!**

To get feedback on how long the training takes, the code in the cell below contains a "timer function". Specifically, the last line of code reads:

```text
print(f"Elapsed time: {hms_string(elapsed_time)}")
```
This code snippet prints out how long the training required.

If you weren't careful and skipped over running the cell that defined the custom function `hms_string()` at the start of this lesson, your model will train for an hour but **FAIL** at the end with an error message.

Since every time you use a Colab GPU **costs you money**, don't waste your time and _money_ by running this code cell if you didn't run _all_ of the code cells above before you run the next one.

**You have been warned.**

In [ ]:
# Example 3 - Step 8: Train Neural Network

import time
import tensorflow as tf

# Set number of epochs
EPOCHS=10

# Set Patience
PATIENCE=5

# ------------------------------------------------------------------------
# 1️⃣  Start training
# ------------------------------------------------------------------------
print(f"-- Training (classification) is starting for {EPOCHS} epochs----------------------------")
start_time = time.time()

# Add more optimization parameters
history_244 = ResNet50_model_244.fit(
    eg_train_gen,
    epochs=EPOCHS,
    validation_data=eg_val_gen,
    steps_per_epoch=len(eg_train_gen),
    validation_steps=len(eg_val_gen),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=PATIENCE, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-7)
    ],
)

# Record end time
elapsed_time = time.time() - start_time

# Print elapsed time
print(f"Elapsed time: {hms_string(elapsed_time)}")

If the code is correct you should see something similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image27D.png)

With Colabs best GPU accerator (`A100`) training for 20 epochs required about 35 min to complete 10 epochs.


------------------------------------------

### **What the training output tells you**

Here is an itemtize explanation of each item is the particular line of training output:
```text
Epoch 4/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 203s 932ms/step - accuracy: 0.5181 - loss: 1.1525 - val_accuracy: 0.5337 - val_loss: 1.1151 - learning_rate: 1.0000e-04
```

* **Epoch 4/20** :  This was the 4th epoch out of a 10-Epoch training session.

* **218/218** : A total of 218 out of 218 **batches** were processed in this epoch.

Remember that there are **13,933** images in the training set and we have set the batch size = 64. If we divide 13,933 images into 64 batches, the model will have process **218** batches to see all of the training images.

* **203s** - Total time for the epoch to complete in seconds (~3 minutes).

* **932ms/step** - Average time per batch in milliseconds (~ 1 second).

* **accuracy: 0.5181** - The accuracy of the model for classifying **_training_** images during this epoch.

* **loss: 1.1525** - The training loss for this epoch

* **val_accuracy: 0.5337** - The accuracy of the model in classifying **_validation_** images during this epoch

* **val_loss: 1.1151** - The validation loss for this epoch.

* **learning_rate: 1.0000e-04** = The learning rate for this epoch.
---------------------

### Example 3 - Step 11: Plot Training/Validation Accuracy

A **Training-versus-Validation Accuracy Plot** provides a quick visual gauge of how well a model is learning and generalizing. As training progresses, a rising training accuracy shows that the model is fitting the data, while the validation accuracy tracks performance on unseen samples.

When both curves rise together and plateau, the model is likely well-balanced. If training accuracy climbs while validation accuracy lags or dips, the model is over-fitting; conversely, if both stay low, the model is under-fitting or too simple.

The shape of the curves also reveals training issues—sharp jumps or oscillations can signal an inappropriate learning rate, and noisy validation performance may indicate label noise or class imbalance. Thus, the plot informs decisions about early stopping, regularisation, architecture changes, or hyper-parameter tuning.


#### **Interpreting Common Patterns**

| Curve Shape | Interpretation | Suggested Action |
|-------------|----------------|------------------|
| **Both curves rise together and plateau at high accuracy** | Good fit & generalisation. | Continue training if you want to squeeze a bit more. |
| **Training rises, validation rises then drops** | Over‑fitting. | Add regularisation, dropout, data augmentation, or stop early. |
| **Both curves rise slowly and stay low** | Under‑fitting. | Increase model capacity, train longer, or reduce regularisation. |
| **Validation lags behind training by a fixed margin, but both improve** | Model learns but generalises less well. | Consider a larger training set, better augmentation, or a different architecture. |
| **Validation fluctuates wildly** | High variance due to small batch or high learning rate. | Reduce learning rate, increase batch size, or use a learning‑rate scheduler. |



In [ ]:
# Example 3 = Step 11: Plot Training/Validation Accuracy

import numpy as np
import matplotlib.pyplot as plt

# Pull the metrics
eg_val_acc = history_244.history.get('val_accuracy')
eg_train_acc = history_244.history.get('accuracy')

# --- Find the epoch with the highest validation accuracy -------------
# np.argmax returns the index (0‑based). Add 1 if you want to show it as "epoch 1, 2, …"
best_epoch_idx = np.argmax(eg_val_acc)          # 0‑based index
best_epoch_num = best_epoch_idx + 1          # 1‑based for display

# -----------------------------------------------------------------------

plt.figure(figsize=(8, 4))
plt.plot(eg_val_acc, label='val_accuracy')
plt.plot(eg_train_acc, label='accuracy')

# Vertical line at the best epoch (0‑based index)
plt.axvline(best_epoch_idx, color='r', linestyle='--',
            label=f'Best epoch (epoch {best_epoch_num})')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training / Validation Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)

# Optional: annotate the exact accuracy value at the best epoch
best_eg_val_acc = eg_val_acc[best_epoch_idx]
plt.text(best_epoch_idx, best_eg_val_acc,
         f'{best_eg_val_acc:.4f}',
         va='bottom', ha='right', color='r', fontsize=9)

plt.tight_layout()
plt.show()


If the code is correct you should see something similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image28D.png)

Alternatively, the output might look more similar to this one

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image55C.png)

Either graph is acceptable.


### Example 3 - Step 12: Save Model to GDrive

Run the next cell to save your retrained `ResNet50_model_244` model to your GDrive.

**IMPORTANT NOTE** You will be using this saved Keras model in a latter class lesson so make sure **not** to delete it from your GDrive!

In [ ]:
# Example 3 - Step 10

import os
import shutil
import tensorflow as tf

# --------------------------------------------------------------
# 1️⃣  Mount Google Drive (do this only once per session)
# --------------------------------------------------------------
# from google.colab import drive
# drive.mount('/content/drive')

# --------------------------------------------------------------
# 2️⃣  Define the names / paths
# --------------------------------------------------------------
model_name     = "ResNet50_model_244"                    # model object name
gdrive_dir     = f"/content/drive/My Drive/{model_name}"  # folder on Drive
gdrive_file    = f"{gdrive_dir}.keras"                    # the file we want to keep

local_dir      = f"/content/{model_name}"                 # the *local* folder you want to delete
local_file     = f"{local_dir}.keras"                     # if you saved a single file locally

# --------------------------------------------------------------
# 3️⃣  Make sure the Drive folder exists
# --------------------------------------------------------------
os.makedirs(gdrive_dir, exist_ok=True)

# --------------------------------------------------------------
# 4️⃣  Save the model *on* Drive (kept forever)
# --------------------------------------------------------------
ResNet50_model_244.save(gdrive_file)   # <-- this writes the file into /content/drive/My Drive/

# --------------------------------------------------------------
# 5️⃣  OPTIONAL: Verify the Drive copy exists
# --------------------------------------------------------------
print("Drive copy present:", os.path.exists(gdrive_file))
!ls ./drive/MyDrive

If the code is correct you should see something similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image54C.png)

------------------------------------------


# **EXERCISE 3**

In **Exercise 3** you are going to re-analyze the Diabetic Retinopathy data set using `ResNet101` in conjunction with larger retinal images that are 512 X 512 pixels.

**`ResNet-101`** was built to give a **deeper**, **larger-receptive-field** backbone that can be trained stably thanks to residual connections. That extra depth—and the corresponding increase in receptive field and representational capacity—makes the network particularly suited to the high-resolution, high-complexity images that dominate modern clinical imaging datasets (e.g., whole-slide pathology, high‑res CT/MRI, retinal photographs).  

#### **1. Depth matters for large, complex images**

| Aspect                       | Why it matters in clinical imaging                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------------|
| **Receptive field**         | • Deeper network ⇒ larger receptive field.<br>• Captures micro‑details (nuclei, capillaries) *and* macro‑context (lobule shape). |
| **Hierarchical feature extraction** | • Early layers learn edges & textures.<br>• Deeper layers combine them into complex motifs.<br>• Needed for multi‑level abstraction (malignant vs. normal). |
| **Parameter efficiency**    | • Residual blocks add depth without exploding the number of weights.<br>• 101‑layer stack packs more power while staying manageable. |


#### **2. Residual connections: the training engine that makes depth feasible**

* **Vanishing / exploding gradients** are especially problematic when you try to back‑propagate through dozens of nonlinear layers. Residual connections add a *shortcut* that lets the gradient flow almost unchanged from the output back to the input, so deeper models can be trained from scratch (or fine‑tuned on a new domain) without catastrophic loss of signal.
* In medical-image research, practitioners often *fine-tune* ImageNet‑pretrained models on limited clinical data. Because residual connections preserve gradient flow, a 101‑layer network can be adapted with fewer epochs and fewer data points than a comparable plain CNN.

#### **3. ResNet‑101 and typical clinical datasets**

| Dataset type              | Typical image size                                    | Typical complexity                                   | ResNet‑101 benefits                                                                                                                                                 |
|---------------------------|-------------------------------------------------------|-------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Whole‑slide pathology     | 10k‑40k px × 10k‑40k px (usually tiled to 512×512)   | Ultra‑fine texture + large‑scale architecture        | • Deep residual layers capture both micro‑ and macro‑features.<br>• Receptive field ≈ 400 px after 5 stages                                                      |
| Chest CT                  | 512×512 or larger                                    | Subtle texture changes, large organs                 | • Deeper encoder gives more context for slice‑to‑slice variation                                                                                                      |
| Retinal fundus            | 5000×5000 px (often cropped to 1024×1024)            | Fine vascular patterns + large vessels               | • Extra depth boosts sensitivity to minute capillary changes                                                                         
> **Bottom line**: ResNet‑101 was *not* invented *solely* for medical imaging, but its architectural traits (depth, residual links, large receptive field) make it a natural fit for the high‑resolution, highly variable images found in clinical datasets.

#### **4. Practical use-cases that illustrate the point**

1. **Whole‑slide analysis**  
   *U‑Net + ResNet‑101 encoder* is the standard backbone for segmentation of tumor vs. normal tissue. The encoder’s depth allows the network to produce rich multi‑scale feature maps that are essential for accurate boundary delineation.

2. **Histopathology classification**  
   A 101‑layer residual network fine‑tuned on 2048×2048 patches yields a 2‑fold improvement in AUC over a 50‑layer variant, especially on datasets where subtle nuclear pleomorphism is the key discriminant.

3. **Radiology reporting**  
   ResNet‑101 pre‑trained on ImageNet is used as the feature extractor for a downstream transformer that produces radiology reports. The larger depth improves the semantic quality of the extracted features, translating into more coherent, clinically relevant reports.

#### **5. What the “larger image size” means**

When people say ResNet‑101 was created “in part to analyze larger image sizes,” they're really referring to two intertwined design goals:

1. **Scale-aware receptive fields** - As the input resolution grows, the network’s receptive field must grow proportionally so that each unit still sees a meaningful portion of the image. Deeper networks naturally achieve this.

2. **Robust training** - High-resolution images contain many more pixels, which can amplify the effects of noise, small mis-alignments, or class imbalance. Residual learning keeps gradients stable, allowing the model to learn meaningful patterns from a massive pixel set without **over-fitting** or getting stuck.

---

#### **TL;DR**

ResNet‑101’s 101 layers give it a large receptive field and the capacity to learn complex, hierarchical features, which is vital for interpreting the fine‑grained details and global context in large medical images. Residual connections make training such a deep network feasible, allowing it to be fine‑tuned on the often limited clinical datasets that are typical in healthcare. That combination is why the architecture was chosen, and later adopted, for many high‑resolution clinical image‑analysis tasks.

#### **Image Size Comparison**

You should realize that `512×512×3` image is more than **4X larger** than a `244×244×3` image. Here's the math:

\begin{aligned}
512 \times 512 \times 3 &= 786{,}432 \\
244 \times 244 \times 3 &= 178{,}608 \\
\frac{786{,}432}{178{,}608} &\approx 4.4
\end{aligned}


So, the **512x512x3** image is approximately **4.4 times larger** than the **244×244×3** image in terms of pixel data. That additional detail might be critical for "seeing" small differences between the images in different diabetic retinal classes.

### **Exercise 3 - Step 1: Set ENVIRONMENTAL VARIABLES**

In the cell below write the code to define a your `ENVIRONMENTAL VARIABLES`. It is important to make the all of the following changes exactly as suggested.

**Code Hints:**

1. Change this line of code
```text
DOWNLOAD_SOURCE = URL+"diabetic_retinopathy_train_244.zip"
```
to read
```text
DOWNLOAD_SOURCE = URL+"diabetic_retinopathy_train_512.zip"
```
2. Change this line of code
```text
EXTRACT_TARGET = os.path.join(PATH,"retinopathy_244")
```
to read
```text
EXTRACT_TARGET = os.path.join(PATH,"retinopathy_512")
```
3. Change this line of code
```text
SOURCE = os.path.join(EXTRACT_TARGET, "train_244")
```
to read
```text
SOURCE = os.path.join(EXTRACT_TARGET, "train_512")
```



In [ ]:
# Insert your code for Exercise 3 - Step 1 here



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image37C.png)

### **Exercise 3 - Step 2: Download and Extract Image Data**

In the cell below write the code to (1) create the necessary directories, (2) download the Zip file containing the image data, (3) extract this data into the appropiate folder and (4) verify the sucess

**Code Hints:**

You can re-use the code in Example 3 - Step 2 _without_ making any code changes.

**TIME WARNING:** This Zip file is _substantially_ larger than the one you downloaded in Example 3. You can expect it to take approximately **7 minutes** to download and unzip the image files.

In [ ]:
# Insert your code for Exercise 3 - Step 2 here



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image10D.png)


### **Exercise 3 - Step 3: Load Labels for the Training Set**

In the cell below write the code to extract the label information from the file `trainLabel.csv` located in your `./retinopathy_512` folder and create a DataFrame called `ex_raw_df`.

**Code Hints:**

Copy the code from `Example 3 - Step 3` and then change the prefix `eg_` to `ex_` **everywhere** in the code.

In [ ]:
# Insert your code for Exercise 3 - Step 3 here


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image40C.png)

### **Exercise 3 - Step 4: Validate Images**

In the cell below write the code to check whether an actual retinal image file exists for each image label in the DataFrame `ex_raw`.  

**Code Hints:**

Make sure to change the prefix `eg_` to `ex_` in the print statements are the bottom of the code copied from Example 3 - Step 4.

In [ ]:
# Insert your code for Execise 3 - Step 4 here



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image29D.png)

### **Exercise 3 - Step 5: Split Images into Training and Validation Sets**

In the cell below write the code to split your retinal images into a training set and a validation set with 80% of the images going into the training set. After splitting, print out the number images in both sets and a short sample of both the training set and the validation set.

**Code Hints:**

Copy the code from `Example 3 - Step 5` and change the prefix `eg_` to `ex_`.

In [ ]:
# Insert your code for Exercise 3 - Step 5 here



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image30D.png)

### **Exercise 3 - Step 6: Final Check for Valid Images**

In the cell below write the to make a final check that all the image files are valid.

**Code Hints:**

Make sure to change the prefix `eg_` to `ex_` every where in the code that you copied from Example 3 - Step 6.

In [ ]:
# Insert your code for Exericse 3 - Step 6 here



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image31D.png)

### **Exercise 3 - Step 7: Create Image Generator**

In the cell below write the code to create an Image Generator using the `Tensorflow/Keras` utility `ImageDataGenerator`.

**Code Notes:**

1. Is is _very_ important that you change your image size.

Specifically you must change this code snippet:
```text
# Specify Image Size
IMG_W, IMG_H = 244, 244
```
to read as
```text
# Specify Image Size
IMG_W, IMG_H = 512, 512
```
The main objective of **Exercise 3** is to analyze larger retinal imagee (512 X 512 pixels) to see if that will improve classification accuracy.

2. Make sure to change:
```text
eg_train_gen = eg_train_datagen.flow_from_dataframe(
    dataframe=eg_train_df,  # Labels for training images
```
to read as
```text
ex_train_gen = ex_train_datagen.flow_from_dataframe(
    dataframe=ex_train_df,  # Labels for training images
```

3. Make sure to change the name of the DataFrame holding the labels:
```text
eg_val_gen = eg_eval_datagen.flow_from_dataframe(
    dataframe=eg_val_df,   # Labels for the validation images
```
to read as
```text
ex_val_gen = ex_val_datagen.flow_from_dataframe(
    dataframe=ex_val_df,   # Labels for the validation images
```

In [ ]:
# Insert your code for Exercise 3 - Step 7 here



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image32D.png)

### **Exercise 3 - Step 8: Check Class Distribution**

In the cell below write the code to generate a bar graph showing the number of images in each of the 5 output classes.

**Code Hints:**

Copy the code from `Example 3 - Step 6` and change the prefix `eg_` to `ex_` everywhere that it occurs.



In [ ]:
# Insert your code for Exercise 3 - Step 8 here




If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image12D.png)

### **Exercise 3 - Step 9: Setup `ResNet101` Base Network**

In the cell below write the code to setup your base network using `ResNet101` instead of `ResNet50`. Called your new model `ResNet101_model_512`.

**Code Hints:**

1. Change this line of code
```text
from tensorflow.keras.applications import ResNet50
```
to read
```text
from tensorflow.keras.applications import ResNet101

2. Change this line of code
```text
base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_H, IMG_W, 3))
```
to read
```text
base = ResNet101(weights='imagenet', include_top=False, input_shape=(IMG_H, IMG_W, 3))
```
2. Change this line of code
```text
ResNet50_model_244 = models.Model(inputs=base.input, outputs=predictions)
```
to read
```text
ResNet101_model_512 = models.Model(inputs=base.input, outputs=predictions)
```
3. Change this line of code
```text
ResNet50_model_244.compile(
```
to read
```text
ResNet101_model_512.compile(
```





In [ ]:
# Example 3 - Step 9: Setup and Compile Neural Network



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image44C.png)

### **Exercise 3 - Step 10: Train Neural Network**

In the cell below write the code to train your neural network `ResNet101_model_512`.

#### **Please Read this Carefully**

The `ResNet101_model_512` is by far the largest neural network that you will be asked to train in this course. Due to the large size of the images (512x512 pixels), training this model for even `10` epochs will require more than **2 hours!**

To make this lesson more "student friendly", you are free to set your number of epochs to a **minumum value** of `2` and still get full credit.

Just be aware that even 2 epochs will probably take more than 1/2 hour to run. If you don't run at least 2 epochs you will lose points.

**Code Hints:**

Change this line of code
```text
history_244 = ResNet50_model_244.fit(
```
to read
```text
history_512 = ResNet101_model_512.fit(
```

Make sure to change the prefix `eg_` to `ex_` everywhere is occurs in the code.

In [ ]:
# Insert your code for Exercise 3 - Step 10 here



If the code is correct you should see something similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image33D.png)


### **Exercise 3 - Step 11: Plot Training History**

In the cell below write the code to plot the training history of your `ResNet101_model_512`.

Copy the code in Example 3 - Step 9 into the cell below.

**Code Hints:**

1. Change this line of code
```text
eg_val_acc = history_244.history.get('val_accuracy')
```
to read
```text
ex_val_acc = history_512.history.get('val_accuracy')
```
2. Change this line of code
```text
eg_train_acc = history_244.history.get('accuracy')
```
to read
```text
ex_train_acc = history_512.history.get('accuracy')
```

Change the prefix `eg_` to `ex_` everywhere in the copied code.

In [ ]:
# Insert your code for Exercise 3 - Step 11 here



If the code is correct you should see something similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image34D.png)


Alternatively, you might see something more similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image51C.png)

### **Analysis**

1. **Training vs Validation Accuracy:**

* Both curves show a general upward trend, indicating that the model is learning over time.
* The training accuracy is consistently higher than the validation accuracy, which is typical and suggests the model is fitting the training data well.

2. **Best Epoch at 9:**

* Although the x-axis only goes up to 8, the best validation accuracy is noted at epoch 9. This might imply that the training continued beyond what's shown in the plot, or the plot is cropped.
* The peak validation accuracy of **`0.7634`** suggests a decent generalization performance.

3. **Possible Overfitting:**

* If the training accuracy continues to rise while validation accuracy plateaus or drops (not fully visible here), it could indicate overfitting. However, from the visible data, the gap between training and validation accuracy is not extreme.

### **Exercise 3 - Step 12: Save Model to GDrive**

In the next cell write the code to save your `ResNet101_model_512` to your GDrive.

**Code Hints:**

Change this line of code
```text
model_name     = "ResNet50_model_244"                    # model object name
```
to read
```text
model_name     = "ResNet101_model_512"                    # model object name
```

In [ ]:
# Insert your code for Example 3 - Step 12 here



If the code is correct you should see something similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image53C.png)

Your `ResNet101_model_512` is now safely stored on your GDrive.

If you wanted to, you could easily copy your model back from your GDrive into you current Google Colab directory and use it analyze new retinal fundus images, looking for signs of diabetic retinopathy. Since your model is already trained, you could analyze the retinal images from your new patient as he/she sits in your office.

Always keep in mind a common saying among members of the ML/AI community:

> **_“Train-once, deploy-many”_**

**IMPORTANT NOTE:** Make sure **not** to erase your saved file from your GDrive. You will need to use it later in a different class lesson.

## **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Class_03_2.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.

## **The Diabetic Retinopathy Dataset**

The **Diabetic Retinopathy Dataset** used in this lesson was part of a 2015 Kaggle Competition.

Here is a summary of the winners and what they did to win this competition.

### Winner of the 2015 Kaggle Diabetic Retinopathy Detection competition
**Team**: *o_O* (Mathis Antony & Stephan Brüggemann)  
**Score**: 0.845 weighted quadratic‑weighted Kappa (private leaderboard)  
**Public leaderboard**: 7th place (5.8 k K‑appa)

| Item | Details | Source |
|------|---------|--------|
| **Winning team** | “o_O” (Mathis Antony & Stephan Brüggemann) | 5.8 k K‑appa on the private leaderboard, 7 th place on the public leaderboard【9†L18-L23】 |
| **Overall performance** | 0.845 weighted quadratic weighted Kappa (private leaderboard) | 5.8 k K‑appa, 0.845 score【9†L19-L22】 |
| **Core architecture** | Two custom 2‑D convolutional nets (Net A & Net B) with a **per‑patient blending network** (Table 2) | 13‑25 | 12‑15 |
| **Training framework** | Lasagne + nolearn (Theano) | 10‑11 |
| **Image size** | 128 × 128, 256 × 256 and 512 × 512 (large color images, cropped to remove background) | 21‑24 |
| **Pre‑training strategy** | *First* train a small network on 128‑pixel images.  Weights are then used to initialise an intermediate‑size network (trained on 256 px) and finally a 512‑pixel network.  Orthogonal initialization for all weights. | 105‑108 |
| **Data augmentation** | Translation, stretching, rotation, flipping, colour jitter; per‑channel zero‑mean/unit‑variance scaling; 112/224/448 output sizes for 128/256/512 input images. | 115‑121 |
| **Class imbalance handling** | Dynamic resampling: oversample rare classes initially, then gradually reduce; resampling weights \((1.36, 14.4, 6.64, 40.2, 49.6)\) → \((1,2,2,2,2)\). | 82‑96 |
| **Training schedule** | Nesterov momentum with a fixed learning‑rate schedule over 250 epochs; learning rates 0.003 (epoch 0) → 0.00003 (epoch 150); L2 weight decay 0.0005; dropout after convolution and dense layers. | 31‑36, 70‑73 |
| **Loss & objective** | Mean‑squared‑error regression (output thresholded at (0.5,1.5,2.5,3.5) to obtain integer grades). | 75‑77 |
| **Blending network** | Input: mean & std of the RMSPool layer over 50 augmentations for each eye (µ,σ) + eye‑side indicator; 8193‑input → Dense 32 → Maxout 16 → Dense 32 → Maxout 16; Adam optimiser with a schedule (5 e‑4 → 5 e‑7). | 148‑156, 158‑166 |
| **Final ensemble** | Average of the two conv‑net predictions, blended with the patient‑level network; score 0.845 (private) vs 0.824 (no per‑patient blend). | 167‑169 |
| **Key design choices that yielded the win** | 1. **Large input resolution** – 512 × 512 (and even 768 × 768 for 0.81 Kappa) to capture micro‑aneurysms. 2. **Stage‑wise pre‑training** – starting from 128 px to 512 px to stabilise training. 3. **Extensive data augmentation & per‑channel normalisation**. 4. **Dynamic resampling** to address class imbalance without a weighted loss. 5. **Per‑patient blending** that aggregates information from both eyes and multiple augmentations. 6. **Ensembling of two independently trained nets**. | 45‑69, 82‑96, 100‑108, 115‑121, 131‑139, 167‑169 |

### **Why their approach won**

The combination of **large‑resolution images**, staged pre‑training, aggressive augmentation, careful imbalance handling, and per‑patient feature blending allowed the o_O model to achieve the highest weighted quadratic‑weighted κ score in the 2015 competition.



## **Lizard Tail**


## **BASIC**

![__](https://upload.wikimedia.org/wikipedia/commons/7/7b/AtariBASIC.png)

# Introduction to BASIC Programming Language

## What is BASIC?

**BASIC** (Beginner's All-purpose Symbolic Instruction Code) is a high-level programming language designed to be easy to learn and use. It was created with the goal of providing access to computing power for students and non-professional programmers, emphasizing simplicity and readability.

BASIC uses straightforward syntax and commands that resemble English, making it an ideal first language for beginners. Over time, it has evolved into many dialects and influenced the development of numerous other programming languages.

---

## **Historical Summary**

#### **Origins**

- **Year Created**: 1964  
- **Creators**: John G. Kemeny and Thomas E. Kurtz  
- **Institution**: Dartmouth College

Kemeny and Kurtz developed BASIC to enable students in fields other than science and mathematics to use computers. At the time, most programming languages were complex and required deep technical knowledge. BASIC was designed to democratize computing.

#### **Key Milestones**

- **1964**: First implementation of BASIC on a GE-225 mainframe at Dartmouth College.
- **1970s**: BASIC became widely adopted on microcomputers, especially with the rise of personal computing.
- **1975**: Microsoft was founded to develop a version of BASIC (Altair BASIC) for the Altair 8800, marking the beginning of Microsoft's software empire.
- **1980s**: Variants like GW-BASIC, QuickBASIC, and Turbo BASIC became popular on MS-DOS systems.
- **1991**: Microsoft introduced Visual Basic, combining BASIC with a graphical user interface (GUI) development environment.
- **2000s–Present**: Modern dialects like VB.NET continue to be used, especially in enterprise and educational settings.

#### **Legacy**

BASIC played a crucial role in the early days of personal computing. It introduced millions of people to programming and laid the foundation for many modern languages. While its use has declined in favor of more powerful and flexible languages, its influence remains significant in the history of computer science.

---

## Example BASIC Code

```basic
10 PRINT "HELLO, WORLD!"
20 END
```

**BASIC (Beginners' All-purpose Symbolic Instruction Code)** is a family of general-purpose, high-level programming languages designed for ease of use. The original version was created by John G. Kemeny and Thomas E. Kurtz at Dartmouth College in 1963. They wanted to enable students in non-scientific fields to use computers. At the time, nearly all computers required writing custom software, which only scientists and mathematicians tended to learn.

In addition to the programming language, Kemeny and Kurtz developed the Dartmouth Time-Sharing System (DTSS), which allowed multiple users to edit and run BASIC programs simultaneously on remote terminals. This general model became popular on minicomputer systems like the PDP-11 and Data General Nova in the late 1960s and early 1970s. Hewlett-Packard produced an entire computer line for this method of operation, introducing the HP2000 series in the late 1960s and continuing sales into the 1980s. Many early video games trace their history to one of these versions of BASIC.

The emergence of microcomputers in the mid-1970s led to the development of multiple BASIC dialects, including Microsoft BASIC in 1975. Due to the tiny main memory available on these machines, often 4 KB, a variety of Tiny BASIC dialects were also created. BASIC was available for almost any system of the era, and became the de facto programming language for home computer systems that emerged in the late 1970s. These PCs almost always had a BASIC interpreter installed by default, often in the machine's firmware or sometimes on a ROM cartridge.

BASIC declined in popularity in the 1990s, as more powerful microcomputers came to market and programming languages with advanced features (such as Pascal and C) became tenable on such computers. By then, most nontechnical personal computer users relied on pre-written applications rather than writing their own programs. In 1991, Microsoft released Visual Basic, combining an updated version of BASIC with a visual forms builder. This reignited use of the language and "VB" remains a major programming language in the form of VB.NET, while a hobbyist scene for BASIC more broadly continues to exist.

**Origin**

John G. Kemeny was the chairman of the Dartmouth College Mathematics Department. Based largely on his reputation as an innovator in math teaching, in 1959 the college won an Alfred P. Sloan Foundation award for \$500,000 to build a new department building. Thomas E. Kurtz had joined the department in 1956, and from the 1960s Kemeny and Kurtz agreed on the need for programming literacy among students outside the traditional STEM fields. Kemeny later noted that "Our vision was that every student on campus should have access to a computer, and any faculty member should be able to use a computer in the classroom whenever appropriate. It was as simple as that."

Kemeny and Kurtz had made two previous experiments with simplified languages, DARSIMCO (Dartmouth Simplified Code) and DOPE (Dartmouth Oversimplified Programming Experiment). These did not progress past a single freshman class. New experiments using Fortran and ALGOL followed, but Kurtz concluded these languages were too tricky for what they desired. As Kurtz noted, Fortran had numerous oddly formed commands, notably an "almost impossible-to-memorize convention for specifying a loop: DO 100, I = 1, 10, 2. Is it '1, 10, 2' or '1, 2, 10', and is the comma after the line number required or not?"

Moreover, the lack of any sort of immediate feedback was a key problem; the machines of the era used batch processing and took a long time to complete a run of a program. While Kurtz was visiting MIT, John McCarthy suggested that time-sharing offered a solution; a single machine could divide up its processing time among many users, giving them the illusion of having a (slow) computer to themselves.[8] Small programs would return results in a few seconds. This led to increasing interest in a system using time-sharing and a new language specifically for use by non-STEM students.

Kemeny wrote the first version of BASIC. The acronym BASIC comes from the name of an unpublished paper by Thomas Kurtz.The new language was heavily patterned on FORTRAN II; statements were one-to-a-line, numbers were used to indicate the target of loops and branches, and many of the commands were similar or identical to Fortran. However, the syntax was changed wherever it could be improved. For instance, the difficult to remember DO loop was replaced by the much easier to remember FOR I = 1 TO 10 STEP 2, and the line number used in the DO was instead indicated by the NEXT I. Likewise, the cryptic IF statement of Fortran, whose syntax matched a particular instruction of the machine on which it was originally written, became the simpler IF I=5 THEN GOTO 100. These changes made the language much less idiosyncratic while still having an overall structure and feel similar to the original FORTRAN.

The project received a $300,000 grant from the National Science Foundation, which was used to purchase a GE-225 computer for processing, and a Datanet-30 realtime processor to handle the Teletype Model 33 teleprinters used for input and output. A team of a dozen undergraduates worked on the project for about a year, writing both the DTSS system and the BASIC compiler. The first version BASIC language was released on 1 May 1964.

Initially, BASIC concentrated on supporting straightforward mathematical work, with matrix arithmetic support from its initial implementation as a batch language, and character string functionality being added by 1965. Usage in the university rapidly expanded, requiring the main CPU to be replaced by a GE-235,[7] and still later by a GE-635. By the early 1970s there were hundreds of terminals connected to the machines at Dartmouth, some of them remotely.

Wanting use of the language to become widespread, its designers made the compiler available free of charge. In the 1960s, software became a chargeable commodity; until then, it was provided without charge as a service with expensive computers, usually available only to lease. They also made it available to high schools in the Hanover, New Hampshire, area and regionally throughout New England on Teletype Model 33 and Model 35 teleprinter terminals connected to Dartmouth via dial-up phone lines, and they put considerable effort into promoting the language. In the following years, as other dialects of BASIC appeared, Kemeny and Kurtz's original BASIC dialect became known as Dartmouth BASIC.

New Hampshire recognized the accomplishment in 2019 when it erected a highway historical marker in Hanover describing the creation of "the first user-friendly programming language".

### **The Rise of BASIC: From Dartmouth Labs to the Personal Computer Revolution**

> **B**asic **A**lgebraic **S**tandard **T**erm (BASIC) is more than a language—it is the catalyst that turned hobbyist curiosity into a worldwide PC industry.  
---

#### **1. Birth of BASIC (1964‑1965)**

| Year | Milestone | Notes |
|------|-----------|-------|
| **1964** | Dartmouth College computer science course | John G. Kemeny & Thomas E. Kurtz design a *simple* language for students who had never coded before. |
| **1965** | Dartmouth BASIC released | Written in 4 KB of machine code for the IBM 704. Emphasis: **clarity**, **interactivity**, and **line‑numbered scripts**. |

### 1.1 Design Goals
- **Accessibility**: Use everyday math notation (`LET`, `PRINT`, `GOTO`).
- **Education**: Allow students to learn programming concepts without deep hardware knowledge.
- **Rapid Prototyping**: Quick translation from idea to executable script.

> *Quote*: “We wanted a language that *could* be used by a person with no background in programming to write useful programs in a reasonable amount of time.” – *Kemeny & Kurtz*

---

#### **2. Early Evolution & Variants**

| Variant | Platform | Key Features | Year |
|---------|----------|--------------|------|
| **Algol‑60‑derived BASIC** | Dartmouth 704, 7090 | First *non‑line‑numbered* mode (interactive) | 1965 |
| **BASIC‑PLUS** | DEC PDP‑8 | Introduced subroutines, `ON GOSUB` | 1967 |
| **Microsoft BASIC** | Altair 8800 (assembly) | Commercialized BASIC; later ported to DOS | 1975 |
| **GW‑BASIC** | IBM PC | Microsoft’s *free* BASIC in DOS 1.0 | 1980 |
| **AppleSoft BASIC** | Apple II | Built‑in `FOR…NEXT`, `READ…DATA` | 1977 |
| **BBC BASIC** | BBC Micro | Graphics, sound, and structured programming support | 1981 |
| **Turbo BASIC** | Turbo Pascal & Turbo C* | Fast interpreter on DOS | 1985 |

*Note:* Most dialects kept the **line‑numbered** format, which was both a strength (easy debugging) and a weakness (inefficient).

---

#### **3. BASIC in the Education & Hobbyist Scene**

##### 3.1 The “BASIC People”
- A generation of programmers grew up writing code in BASIC.
- Encouraged *self‑education* and *problem‑solving* outside formal settings.

##### 3.2 First‑hand Experience
```basic
10 PRINT "HELLO, WORLD!"
20 GOTO 10
```

## **Simplicity and Immediate Feedback**
- **Simplicity**: One line of code, one command, no compilation.  
- **Immediate feedback**: The interpreter echoed output instantly.

#### **3.3 Software & Games**
- Early microcomputer game studios (e.g., Sierra, Origin Systems) started in BASIC.  
- BASIC was the lingua franca for shareware and homebrew games.

#### **4. BASIC Meets the Microcomputer Revolution**

| Microcomputer | BASIC Variant | Impact |
|---------------|---------------|--------|
| Altair 8800 | Altair BASIC (Microsoft) | Sparked the Altair BASIC craze; early PC kit sales. |
| Apple II | Applesoft BASIC | Built‑in BASIC; many users bought the Apple II solely for the built‑in interpreter. |
| Commodore PET | Commodore BASIC | Bundled with all PET models; the first home computer with a full keyboard and monitor. |
| TRS‑80 | Tandy BASIC | Made programming accessible to hobbyists; popular in the “Micro‑computer” market. |
| IBM PC (1981) | Microsoft’s GW‑BASIC | Became part of the default DOS installation, ensuring a ready‑made audience for PC software. |

#### **5. Why BASIC Was Key to the PC Revolution**

| Factor | How BASIC Contributed |
|--------|-----------------------|
| Low Barrier to Entry | Students and hobbyists could write simple programs without mastering machine code. |
| Rapid Development | Interactive loop allowed quick prototyping → more software in the market. |
| Learning Path | BASIC served as a stepping stone to languages like Pascal, C, and later Java/Python. |
| Ecosystem Creation | A huge library of demos, utilities, and games created a “software market” for PCs. |
| Cultural Momentum | The phrase “I wrote a BASIC program” became part of tech folklore, fostering a generation of self‑taught coders. |

#### **6. The Decline of BASIC and Its Legacy**

| Period | Change | Outcome |
|--------|--------|---------|
| Early 1990s | Introduction of C/C++, Visual Basic, and Java | BASIC’s role as mainstream teaching language diminished. |
| Late 1990s | Emphasis on structured programming | Many BASIC dialects replaced by VB or VB.NET for business apps. |
| 2000s‑Present | Python & JavaScript | Python’s readability echoes BASIC’s ethos; it is now the primary teaching language in many curricula. |

**Legacy**

- BASIC’s design principles (interactive, readable syntax) influenced later languages.  
- Microsoft’s Visual Basic introduced the event‑driven GUI paradigm, a staple in modern programming.

#### **7. Quick Reference: Sample BASIC Code**

```basic
REM Simple calculator
10 PRINT "Enter first number: ";
20 INPUT A
30 PRINT "Enter second number: ";
40 INPUT B
50 PRINT "Sum is "; A + B
```